# Задание № 1

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 
import nltk

In [2]:
from collections import Counter

In [3]:
from nltk.tokenize import sent_tokenize

dvach = open('2ch_corpus.txt', encoding = 'UTF-8').read()
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)][:-100]
sentences_perplexity = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)][-100:]

In [4]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

i = 0

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))
    if i >4000:
        break
    i += 1

In [5]:
matrix_dvach = np.zeros((len(bigrams_dvach), 
                   len(unigrams_dvach)))

id2word_dvach_1 = list(unigrams_dvach)
word2id_dvach_1 = {word:i for i, word in enumerate(id2word_dvach_1)}

id2word_dvach_2 = list(bigrams_dvach)
word2id_dvach_2 = {word:i for i, word in enumerate(id2word_dvach_2)}

In [9]:
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix

In [12]:
matrix_dvach = lil_matrix((len(bigrams_dvach), 
                   len(unigrams_dvach)))

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {word:i for i, word in enumerate(id2bigram_dvach)}

for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2

    matrix_dvach[bigram2id_dvach[bigram], word2id_dvach[word3]] =  (trigrams_dvach[ngram]/
                                                                     bigrams_dvach[bigram])

In [13]:
def generate(matrix, id2word, word2id, n=200, start ='<start> <start>'):
    text = []
    current_idx = word2id['<start>']
    
    for i in range(n):
        
        chosen = np.random.choice(list(range(matrix.shape[1])), p=matrix[current_idx].toarray()[0])
        text.append(id2word[chosen])
        
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [14]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach, n=200).replace('<end>', '\n\n'))

скажи 

 снова ты при мб и прочих лулзов от звука в отправляется говно ещё драфт ставите для ссылка порнотред в отправляется непотребство прочее и прочих клованов что ffmpeg файлов кодеки плеер установить воспроизведения и прочих лулзов от звука в отправляется непотребство прочее и прочих лулзов от звука в отправляется непотребство прочее и прочих лулзов от звука в отправляется говно ещё драфт ставите для ссылка порнотред в отправляется непотребство прочее и прочих клованов что ffmpeg файлов кодеки плеер установить воспроизведения и прочих аутистов 

 дохуя стерто ара на видео o поиска для ссылка порнотред в отправляется непотребство прочее и прочих аутистов 

 лол лиц буду дальше без с вечно востребованного иди клона то для ссылка порнотред в отправляется говно ещё драфт ставите для ссылка порнотред в отправляется непотребство прочее и прочих лулзов от звука в отправляется говно ещё драфт ставите для ссылка порнотред в отправляется говно ещё драфт ставите для ссылка порнотред в отправ

In [15]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

In [28]:
for sent in sentences_dvach[:10]:
    prob = []
    for ngram in ngrammer(sent, 3):
        word1, word2, word3 = ngram.split()
        bigram = word1 + ' '+word2
    if ngram in trigrams_dvach and bigram in bigrams_dvach:
            prob.append(np.log(trigrams_dvach[ngram] / bigrams_dvach[bigram]))
  
    print(" ".join(sent), perplexity(prob), '\n')

<start> <start> анимублядский webm-треддля приличных анимублядей и прочих аутистов <end> 1.0 

<start> <start> безграмотное быдло с дубляжом войсовером порнографией и котиками советы мерзких мокрописечников вниманиебляди всех видов и прочее непотребство отправляется в порнотред ссылка для поиска с o уса видео сохраняем кадр правый клик по видео и ищем его на для воспроизведения webm с 10-битным цветом нужно установить плагин vlc и отключить встроенный в браузер плеер media <end> 1.0 

<start> <start> webm <end> 1.0 

<start> <start> enabled false в firefox о кодировании webm доступные кодеки — vp 8 и vp 9 для видео vorbis и opus для звука максимальный размер файла — 10240 кб всех файлов в посте — около 40 мб <end> 1.0 

<start> <start> делать webm можно научиться в вики треда там находится подробная информация о выборе и настройке кодеков на примерах использования консольных утилит ffmpeg vpxenc и mkvmerge <end> 1.0 

<start> <start> неочевидные моменты— libvorbis при указании битрейта

# Задание № 2

1) Существует два способа (указанные в сатье), которыми можно бороться с проблемой несловарных слов ("UNK"):
1. Выбрать фиксированный словарь;
2. С помощью вычисления частоты (а именно - малочастотности) заменять их на UNK.

2) Чтобы словам, которые имеются в слоаре, но находятся в тестовом наборе в невидимом контексте (например, они появляются после слова, после которого они никогда не появлялись при обучении), не приписывалась нулевая вероятность, им добавляется какая-то часть вероятности более частотных слов. Эта модификация называется сглаживанием или дисконтированием (smoothing or discounting).